In [ ]:
#|default_exp doclinks

# doclinks
> Generating a documentation index from a module
- order: 5

In [ ]:
#|export
from nbdev.config import *
from nbdev.maker import *
from nbdev.export import *
from nbdev.imports import *

from fastcore.script import *
from fastcore.utils import *
from fastcore.meta import delegates
from fastcore.net import urlread

import ast,builtins,contextlib
import pkg_resources,importlib

from astunparse import unparse
from io import BytesIO
from collections import defaultdict
from pprint import pformat
from urllib.parse import urljoin
from functools import lru_cache
from types import ModuleType

In [ ]:
#|hide
from IPython.display import Markdown,display
from unittest.mock import patch as xpatch
from fastcore.test import *
from pdb import set_trace
from importlib import reload
from nbdev.showdoc import show_doc

## Create the module index

In [ ]:
#|export
def _sym_nm(klas, sym): return f'{unparse(klas).strip()}.{sym.name}'

def _binop_leafs(bo, o):
    "List of all leaf nodes under a `BinOp`"
    def f(b): return _binop_leafs(b, o) if isinstance(b, ast.BinOp) else [_sym_nm(b,o)]
    return f(bo.left) + f(bo.right)

def patch_name(o):
    "If `o` is decorated with `patch` or `patch_to`, return its class-prefix name"
    if not isinstance(o, (ast.FunctionDef,ast.AsyncFunctionDef)): return o.name
    d = first([d for d in o.decorator_list if decor_id(d).startswith('patch')])
    if not d: return o.name
    nm = decor_id(d)
    if nm=='patch': 
        a = o.args.args[0].annotation
        if isinstance(a, ast.BinOp): return _binop_leafs(a, o)
    elif nm=='patch_to': a = d.args[0]
    else: return o.name
    return _sym_nm(a,o)

In [ ]:
def _test_patch(code): return patch_name(ast.parse(code).body[0])
s = "@patch\ndef _f(self:_T): ..."
test_eq('_T._f', _test_patch(s))

In [ ]:
s = "@patch_to(_T)\ndef _g(self): ..."
test_eq('_T._g', _test_patch(s))

In [ ]:
# Get all patched classes when patching with a union
s = "@patch\ndef _f(self:_T|_U|_V): ..."
test_eq(_test_patch(s), ['_T._f', '_U._f', '_V._f'])

In [ ]:
#|export
def _iter_py_cells(p):
    "Yield cells from an exported Python file."
    p = Path(p)
    cells = p.read_text(encoding='utf-8').split("\n# %% ")
    has_cell_number = get_config().cell_number
    for cell in cells[1:]:
        top,code = cell.split('\n', 1)
        try:
            if has_cell_number:
                *nb,idx = top.split()
                nb = ' '.join(nb)
                idx = int(idx)
            else:
                nb = top
                idx = None
        except ValueError: raise ValueError(f"Unexpected format in '{p}' at cell:\n```\n# %% {cell.strip()}.\n```\n"
                                            "The expected format is: '# %% {nb_path} {cell_idx}'.")
        nb_path = None if nb=='auto' else (p.parent/nb).resolve()  # NB paths are stored relative to .py file
        if code.endswith('\n'): code=code[:-1]
        yield AttrDict(nb=nb, idx=idx, code=code, nb_path=nb_path, py_path=p.resolve())

In [ ]:
#|hide
# it = _iter_py_cells('../../nbdev/sync.py')
# display(next(it))
# display(next(it))

In [ ]:
#|export
def _nbpath2html(p): return p.with_name(re.sub(r'^\d+[a-zA-Z0-9]*_', '', p.name.lower())).with_suffix('.html')

In [ ]:
#|hide
test_eq(_nbpath2html(Path('foo/bar/012abc_my.notebook.ipynb')), Path('foo/bar/my.notebook.html'))

In [ ]:
#|export
def _get_modidx(py_path, code_root, nbs_path):
    "Get module symbol index for a Python source file"
    cfg = get_config()
    rel_name = py_path.resolve().relative_to(code_root).as_posix()
    mod_name = '.'.join(rel_name.rpartition('.')[0].split('/'))  # module name created by py_path

    _def_types = ast.FunctionDef,ast.AsyncFunctionDef,ast.ClassDef
    d = {}
    for cell in _iter_py_cells(py_path):
        if 'auto' in cell.nb: continue
        loc = _nbpath2html(cell.nb_path.relative_to(nbs_path))

        def _stor(nm):
            for n in L(nm): d[f'{mod_name}.{n}'] = f'{loc.as_posix()}#{n.lower()}',rel_name
        for tree in ast.parse(cell.code).body:
            if isinstance(tree, _def_types): _stor(patch_name(tree))
            if isinstance(tree, ast.ClassDef):
                for t2 in tree.body:
                    if isinstance(t2, _def_types): _stor(f'{tree.name}.{t2.name}')
    return {mod_name: d}

In [ ]:
#|hide
# _get_modidx(get_config().lib_path/'sync.py', get_config().lib_path, get_config().nbs_path)

In [ ]:
#|export
def _build_modidx(dest=None, nbs_path=None, skip_exists=False):
    "Create _modidx.py"
    if dest is None: dest = get_config().lib_path
    if not Path(dest).exists(): return
    nbs_path = Path(nbs_path or get_config().nbs_path).resolve()
    if os.environ.get('IN_TEST',0): return
    idxfile = dest/'_modidx.py'
    if skip_exists and idxfile.exists(): return
    with contextlib.suppress(FileNotFoundError): idxfile.unlink()
    if idxfile.exists(): res = exec_local(idxfile.read_text(encoding='utf-8'), 'd')
    else: res = dict(syms={}, settings={}) 
    res['settings'] = {k:v for k,v in get_config().d.items()
                       if k in ('doc_host','doc_baseurl','lib_path','git_url','branch')}
    code_root = dest.parent.resolve()
    for file in globtastic(dest, file_glob="*.py", skip_file_re='^_', skip_folder_re=r"\.ipynb_checkpoints"):
        res['syms'].update(_get_modidx((dest.parent/file).resolve(), code_root, nbs_path=nbs_path))
    idxfile.write_text("# Autogenerated by nbdev\n\nd = "+pformat(res, width=140, indent=2, compact=True)+'\n')

In [ ]:
# _build_modidx()

In [ ]:
#| hide
import shutil

In [ ]:
#| hide
#| eval:false
with contextlib.suppress(FileNotFoundError): shutil.rmtree('tmp')

everything_fn = '../../tests/01_everything.ipynb'
nb_export('../../tests/00_some.thing.ipynb', 'tmp')
nb_export(everything_fn, 'tmp')

mod_fn = Path('tmp')
_build_modidx(mod_fn, nbs_path=Path('../../tests/').resolve())

d = exec_import('tmp._modidx', 'd')['d']
d['syms']['tmp.some.thing']

{'tmp.some.thing.h_n': ('everything.html#h_n', 'tmp/some/thing.py')}

## Export a notebook

In [ ]:
#|export
@delegates(globtastic)
def nbglob(path=None, skip_folder_re = '^[_.]', file_re=r'.*\.ipynb|.*\.qmd', skip_file_re='^[_.]', key='nbs_path', as_path=False, **kwargs):
    "Find all files in a directory matching an extension given a config key."
    path = Path(path or get_config()[key])
    recursive=get_config().recursive
    res = globtastic(path, file_re=file_re, skip_folder_re=skip_folder_re,
                     skip_file_re=skip_file_re, recursive=recursive, **kwargs)
    return res.map(Path) if as_path else res

In [ ]:
#|export
def nbglob_cli(
    path:str=None, # Path to notebooks
    symlinks:bool=False, # Follow symlinks?
    file_glob:str=None, # Only include files matching glob
    file_re:str=r'.*\.ipynb|.*\.qmd', # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str='^[_.]', # Skip files matching regex
    skip_folder_re:str = '^[_.]'): # Skip folders matching regex
    "Find all files in a directory matching an extension given a config key."
    return nbglob(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re, folder_re=folder_re,
                  skip_file_glob=skip_file_glob, skip_file_re=skip_file_re, skip_folder_re=skip_folder_re)

In [ ]:
#|export
@call_parse
@delegates(nbglob_cli)
def nbdev_export(
    path:str=None, # Path or filename
    procs:Param("tokens naming the export processors to use.", nargs="*", choices=optional_procs())="black_format",
    **kwargs):
    "Export notebooks in `path` to Python modules"
    if os.environ.get('IN_TEST',0): return
    if not is_nbdev(): raise Exception('`nbdev_export` must be called from a directory within a nbdev project.')
    if procs:
        import nbdev.export
        procs = [getattr(nbdev.export, p) for p in L(procs)]
    files = nbglob(path=path, as_path=True, **kwargs).sorted('name')
    for f in files: nb_export(f, procs=procs)
    add_init(get_config().lib_path)
    _build_modidx()

`procs` names the optional processors you wish to run on the exported cells of your notebook.

N.B.: the `black_format` processor is passed in by default. But it is a no-op, unless `black_formatting=True` is set in your `settings.ini` configuration. You can omit it from `nbdev_export` on the command line by passing in `--procs`.

## Construct Index

In [ ]:
#|export
typs = 'module','class','method','function'
bset = set(dir(builtins))

In [ ]:
#|export
def create_index(url, pre=None):
    "Create a documentation index from a sphinx inventory file at `url`, with optional prefix `pre`"
    try: from sphinx.util.inventory import InventoryFile
    except ImportError: raise ImportError('`sphinx` is a dependency for building indexes. Run `pip install sphinx` to use `create_index`.')
    pre = ifnone(pre, f"{url}/")
    invs = urlread(f'{url}/objects.inv', decode=False)
    idx = InventoryFile.load(stream=BytesIO(invs), uri=pre, joinfunc=urljoin)
    _get = lambda o: {k:v[2] for k,v in idx[f'py:{o}'].items() if k[0]!='_'}
    d = {o:_get(o) for o in typs}
    syms = defaultdict(dict)
    for o in typs:
        for k,v in d[o].items():
            if k.split('.')[0] in bset: k = 'builtins.' + k
            modparts = k.split(".")[:-2 if o=='method' else -1]
            if modparts: syms['.'.join(modparts)][k] = v
    return syms

In [ ]:
url = 'https://docs.python.org/3'
syms = create_index(url)

for b in syms['builtins']:
    b = b.split('.')
    if len(b) != 2: continue
    b = b[1]
    assert b in bset

dict(list(syms['builtins'].items())[:10])

{'builtins.bool': 'https://docs.python.org/3/library/functions.html#bool',
 'builtins.bytearray': 'https://docs.python.org/3/library/stdtypes.html#bytearray',
 'builtins.bytes': 'https://docs.python.org/3/library/stdtypes.html#bytes',
 'builtins.complex': 'https://docs.python.org/3/library/functions.html#complex',
 'builtins.dict': 'https://docs.python.org/3/library/stdtypes.html#dict',
 'builtins.float': 'https://docs.python.org/3/library/functions.html#float',
 'builtins.frozenset': 'https://docs.python.org/3/library/stdtypes.html#frozenset',
 'builtins.int': 'https://docs.python.org/3/library/functions.html#int',
 'builtins.list': 'https://docs.python.org/3/library/stdtypes.html#list',
 'builtins.memoryview': 'https://docs.python.org/3/library/stdtypes.html#memoryview'}

## Query the module index

In [ ]:
#|export
import importlib,ast
from functools import lru_cache

In [ ]:
#|export
def _find_mod(mod):
    mp,_,mr = mod.partition('/')
    spec = importlib.util.find_spec(mp)
    if not spec: return
    loc = Path(spec.origin).parent
    return loc/mr

@lru_cache(None)
def _get_exps(mod):
    "Get the line numbers for function and class definitions in module"
    mf = _find_mod(mod)
    if not mf: return {}
    txt = mf.read_text(encoding='utf-8')
    _def_types = ast.FunctionDef,ast.AsyncFunctionDef,ast.ClassDef
    d = {}
    for tree in ast.parse(txt).body:
        if isinstance(tree, _def_types):
            for t in L(patch_name(tree)): d[t] = tree.lineno
        if isinstance(tree, ast.ClassDef): d.update({tree.name+"."+t2.name: t2.lineno for t2 in tree.body if isinstance(t2, _def_types)})
    return d

def _lineno(sym, fname): return _get_exps(fname).get(sym, None) if fname else None

In [ ]:
#|hide
_get_exps.cache_clear()

# Test _get_exps caching
initial = _get_exps.cache_info()
_ = _get_exps('nbdev/maker.py')  # First call should miss
after_first = _get_exps.cache_info()
_ = _get_exps('nbdev/maker.py')  # Second call should hit
after_second = _get_exps.cache_info()

test_eq(after_first.misses, initial.misses + 1)
test_eq(after_first.hits, initial.hits)
test_eq(after_second.hits, after_first.hits + 1)
test_eq(after_second.misses, after_first.misses)

In [ ]:
#|hide
assert _lineno('urlsend', 'fastcore/net.py')
assert _lineno('ModuleMaker.make', 'nbdev/maker.py')

In [ ]:
#|export
def _qual_sym(s, settings):
    "Get qualified nb, py, and github paths for a symbol s"
    if not isinstance(s,tuple): return s
    nb,py = s
    nbbase = urljoin(settings["doc_host"]+'/',settings["doc_baseurl"])
    nb = urljoin(nbbase+'/', nb)
    gh = urljoin(settings["git_url"]+'/', f'blob/{settings["branch"]}/{py}')
    return nb,py,gh

def _qual_mod(mod_d, settings): return {sym:_qual_sym(s, settings) for sym,s in mod_d.items()}
def _qual_syms(entries):
    settings = entries['settings']
    if 'doc_host' not in settings: return entries
    return {'syms': {mod:_qual_mod(d, settings) for mod,d in entries['syms'].items()}, 'settings':settings}

In [ ]:
#|export
_re_backticks = re.compile(r'`([^`\s]+?)(?:\(\))?`')

In [ ]:
#|hide
test_eq(_re_backticks.findall('test `foo` here'), ['foo'])
test_eq(_re_backticks.findall('test `foo()` here'), ['foo'])
test_eq(_re_backticks.findall('multiple `foo` and `bar()` matches'), ['foo', 'bar'])
test_eq(_re_backticks.findall('no match'), [])
test_eq(_re_backticks.findall('`weird``thing`'), ['weird', 'thing'])
test_eq(_re_backticks.findall('`spaces not allowed `'), [])
test_eq(_re_backticks.findall('`sym.with.dots`'), ['sym.with.dots'])
test_eq(_re_backticks.findall('unicode `αβγ`'), ['αβγ'])
test_eq(_re_backticks.findall('long `extremely.long.module.name.with.many.dots.function`'), 
        ['extremely.long.module.name.with.many.dots.function'])
test_eq(_re_backticks.findall('special chars `foo[bar]` `foo$bar`'), ['foo[bar]', 'foo$bar'])
test_eq(_re_backticks.findall('consecutive `foo``bar``baz`'), ['foo', 'bar', 'baz'])

In [ ]:
#| export
@lru_cache(None)
def _build_lookup_table(strip_libs=None, incl_libs=None, skip_mods=None):
    cfg = get_config()
    if strip_libs is None:
        try: strip_libs = cfg.get('strip_libs', cfg.get('lib_path', 'nbdev').name).split()
        except FileNotFoundError: strip_libs = 'nbdev'
    skip_mods = setify(skip_mods)
    strip_libs = L(strip_libs)
    if incl_libs is not None: incl_libs = (L(incl_libs)+strip_libs).unique()
    entries = {}
    for o in pkg_resources.iter_entry_points(group='nbdev'):
        if incl_libs is not None and o.dist.key not in incl_libs: continue
        try: entries[o.name] = _qual_syms(o.resolve())
        except Exception: pass
    py_syms = merge(*L(o['syms'].values() for o in entries.values()).concat())
    for m in strip_libs:
        if m in entries:
            _d = entries[m]
            stripped = {}
            for mod, dets in _d['syms'].items():
                if mod not in skip_mods:
                    for k,v in dets.items():
                        k = remove_prefix(k,f"{mod}.")
                        if k not in stripped: stripped[k] = v
            py_syms = merge(stripped, py_syms)
    return entries,py_syms

In [ ]:
#|hide
_build_lookup_table.cache_clear()

# Test _build_lookup_table caching
initial = _build_lookup_table.cache_info()
_ = _build_lookup_table()  # First call should miss
after_first = _build_lookup_table.cache_info()
_ = _build_lookup_table()  # Second call should hit
after_second = _build_lookup_table.cache_info()

test_eq(after_first.misses, initial.misses + 1)
test_eq(after_first.hits, initial.hits)
test_eq(after_second.hits, after_first.hits + 1)
test_eq(after_second.misses, after_first.misses)

Let's test out our error handling when one of the entry points throws an error:

In [ ]:
# Create mock entry points
class BadEntryPoint:
    name = 'bad_entry'
    dist = type('Dist', (), {'key': 'bad_lib'})()
    def resolve(self): raise AttributeError("Simulated error")

class GoodEntryPoint:
    name = 'good_entry'
    dist = type('Dist', (), {'key': 'good_lib'})()
    def resolve(self): return {'syms': {}, 'settings': {}}

In [ ]:
# Clear the cache before testing
_build_lookup_table.cache_clear()

# Patch iter_entry_points
with xpatch('pkg_resources.iter_entry_points', return_value=[BadEntryPoint(), GoodEntryPoint()]):
    entries, py_syms = _build_lookup_table()
    
    # Should only contain the good entry
    assert 'bad_entry' not in entries
    assert 'good_entry' in entries
    assert len(entries) == 1
entries

{'good_entry': {'syms': {}, 'settings': {}}}

In [ ]:
#|export
class NbdevLookup:
    "Mapping from symbol names to docs and source URLs"
    def __init__(self, strip_libs=None, incl_libs=None, skip_mods=None, ns=None):
        self.entries,self.syms = _build_lookup_table(strip_libs, incl_libs, skip_mods)
        self.aliases = {n:o.__name__ for n,o in (ns or {}).items() if isinstance(o, ModuleType)}
        
    def __getitem__(self, s): 
        if '.' in s:
            pre,post = s.split('.', 1)
            if pre in self.aliases: s = f"{self.aliases[pre]}.{post}"
        return self.syms.get(s, None)

    def doc(self, sym):
        "Link to docs for `sym`"
        res = self[sym]
        return res[0] if isinstance(res, tuple) else res

    def code(self, sym):
        "Link to source code for `sym`"
        res = self[sym]
        if not isinstance(res, tuple): return None
        _,py,gh = res
        line = _lineno(sym, py)
        return f'{gh}#L{line}'
    
    def _link_sym(self, m):
        l = m.group(1)
        s = self.doc(l)
        if s is None: return m.group(0)
        l = l.replace('\\', r'\\')
        if m.group(0).endswith('()`'): l += '()'
        return rf"[`{l}`]({s})"

    def link_line(self, l): return _re_backticks.sub(self._link_sym, l)

    def linkify(self, md):
        if md:
            in_fence=False
            lines = md.splitlines()
            for i,l in enumerate(lines):
                if l.startswith("```"): in_fence=not in_fence
                elif not l.startswith('    ') and not in_fence: lines[i] = self.link_line(l)
            return '\n'.join(lines)

Indexing returns a link to the symbol's docs, along with the name of the source file the source URL if available.

In [ ]:
c = NbdevLookup()
c['nbdev.doclinks.NbdevLookup']

('https://nbdev.fast.ai/api/doclinks.html#nbdevlookup',
 'nbdev/doclinks.py',
 'https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/doclinks.py')

In [ ]:
show_doc(NbdevLookup.doc)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/doclinks.py#L269){target="_blank" style="float:right; font-size:smaller"}

### NbdevLookup.doc

>      NbdevLookup.doc (sym)

*Link to docs for `sym`*

Here's a test suite that verifies the error handling behavior:

In [ ]:
c.doc('nbdev.doclinks.NbdevLookup')

'https://nbdev.fast.ai/api/doclinks.html#nbdevlookup'

Symbol names are taken from libraries registered using the 'nbdev' entry point. By default, all libraries with this entry point are searched, but full symbol names (including module prefix) are required.

In [ ]:
assert c.doc('numpy.array').startswith('http')
assert not c.doc('numpy.Array')
assert c.doc('NbdevLookup').endswith('#nbdevlookup')
assert not c.doc('array')

Pass `strip_libs` to list libraries which should be available without requiring a module prefix.

In [ ]:
c = NbdevLookup(strip_libs=('nbdev', 'nbdev_numpy'))
assert c.doc('array').startswith('http')

In [ ]:
show_doc(NbdevLookup.code)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/doclinks.py#L274){target="_blank" style="float:right; font-size:smaller"}

### NbdevLookup.code

>      NbdevLookup.code (sym)

*Link to source code for `sym`*

In [ ]:
NbdevLookup().code('fastcore.net.urlsend')

'https://github.com/AnswerDotAI/fastcore/blob/main/fastcore/net.py#LNone'

In [ ]:
show_doc(NbdevLookup.linkify)

---

[source](https://github.com/AnswerDotAI/nbdev/blob/main/nbdev/doclinks.py#L292){target="_blank" style="float:right; font-size:smaller"}

### NbdevLookup.linkify

>      NbdevLookup.linkify (md)

In [ ]:
md = """This is a link to `numpy.array` and to `array()` and to `get_config()` but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `get_config`

```
This isn't a link to `get_config` either
```"""

In [ ]:
#|eval: false
print(NbdevLookup(('nbdev','numpy')).linkify(md))

This is a link to [`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array) and to `array()` and to [`get_config()`](https://nbdev.fast.ai/api/config.html#get_config) but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `get_config`

```
This isn't a link to `get_config` either
```


In [ ]:
# Test code blocks
md = """```python
def foo():
    return `bar`
```"""
assert NbdevLookup().linkify(md) == md

In [ ]:
# Test builtins
md = "`builtins.str.split`"
NbdevLookup().linkify(md)

'[`builtins.str.split`](https://docs.python.org/3/library/stdtypes.html#str.split)'

In [ ]:
# ... now with stripping
md = "`str.split` and `str`"
NbdevLookup('nbdev_stdlib').linkify(md)

'[`str.split`](https://docs.python.org/3/library/stdtypes.html#str.split) and [`str`](https://docs.python.org/3/library/stdtypes.html#str)'

When there is a conflict, the linkification will apply in the order of the stripped libraries and then by alphabetical order. For example, `enumerate` is both a builtin and a function in the threading module. However, since builtins comes first alphabetically, it will take priority

In [ ]:
md = "`enumerate`, `builtins.enumerate` and `threading.enumerate`"
NbdevLookup(('nbdev_stdlib')).linkify(md)

'[`enumerate`](https://docs.python.org/3/library/functions.html#enumerate), [`builtins.enumerate`](https://docs.python.org/3/library/functions.html#enumerate) and [`threading.enumerate`](https://docs.python.org/3/library/threading.html#threading.enumerate)'

We can also take the `find()` function as another instance, it exists as a standard library and in numpy. Therefore, depending on the order of stripped libraries we pass, `find()` will link to either numpy or standard library.

In [ ]:
md = "`find()`"
NbdevLookup(('nbdev_numpy','nbdev_stdlib')).linkify(md)

'[`find()`](https://numpy.org/doc/stable/reference/generated/numpy.char.find.html#numpy.char.find)'

In [ ]:
md = "`find()`"
NbdevLookup(('nbdev_stdlib','nbdev_numpy')).linkify(md)

'[`find()`](https://docs.python.org/3/library/gettext.html#gettext.find)'

You can also use NbdevLookup with import aliases like the following:

In [ ]:
import numpy as np

In [ ]:
NbdevLookup(ns=globals()).linkify('this is an aliased import link `np.array`')

'this is an aliased import link [`np.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array)'

## Export -

In [ ]:
#|eval: false
#|hide
Path('../nbdev/export.py').unlink(missing_ok=True)
nbdev_export()

g = exec_new('import nbdev.export')
assert hasattr(g['nbdev'].export, 'nb_export')
from nbdev._modidx import d
assert d['syms']['nbdev.doclinks']['nbdev.doclinks.NbdevLookup'][0]

JSONDecodeError: Extra data: line 1 column 3 (char 2)